In [1]:
import pandas as pd
import geopandas as gpd
import sqlite3
from shapely.geometry import Point
import folium
from itertools import chain
import numpy as np
import markdown
from pathlib import Path

In [2]:
from kapalo_py.schema_inference import Table, connections, KapaloTables
import kapalo_py.kapalo_map as kapalo_map

In [3]:
kapalo_tables = kapalo_map.read_kapalo_tables(path=Path("tests/sample_data/kapalo_sql/kapalo.sqlite"))
imgs_path = Path("tests/sample_data/kapalo_imgs")

In [4]:
project_map = kapalo_map.create_project_map(
    kapalo_tables, 
    project="Kurikka GTK",
    imgs_path=imgs_path,
)

ERROR:root:No match for image id NO-2021-13.k1 in tests/sample_data/kapalo_imgs
ERROR:root:No match for image id NO-2021-16.k1 in tests/sample_data/kapalo_imgs
ERROR:root:No match for image id NO-2021-17.k2 in tests/sample_data/kapalo_imgs


In [5]:
project_map.save("live-mapping/index.html")